In [4]:
import math
import cv2
from sklearn import neighbors
import numpy as np
import pandas as pd
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
curd = os.getcwd()
try:  
    os.mkdir("{}/known_people".format(curd))
except:
    pass
try:  
    os.mkdir("{}/models".format(curd))
except:
    pass
try:  
    os.mkdir("{}/test".format(curd))
except:
    pass
try:  
    os.mkdir("{}/unknown_pictures".format(curd))
except:
    pass
names=[]

In [11]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [12]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
   
    X = []
    y = []

    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [13]:

def predict(frame, knn_clf=None, model_path=None, distance_threshold=0.6):
    

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    
   # X_img = face_recognition.load_image_file(X_img_path)
    X_img = frame
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


In [14]:
#my version
def show_prediction_labels_on_image(frame, predictions):
    
    #pil_image = Image.open(img_path).convert("RGB")
    pil_image = Image.fromarray(frame).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        
        name = name.encode("UTF-8")

        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    del draw

    #pil_image.show()
    #cv2.imshow("frame",pil_image)
    return np.asarray(pil_image)


In [ ]:
#Train data creator
import cv2
import numpy as np
import time
import os
curd = os.getcwd()
name = input()
path = "{}/known_people/{}".format(curd,name)
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()

count=0
while count<10:
    
    rat, frame = cap.read()
    cv2.imwrite("{}/{}{}.jpg".format(path,name,count), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


cap.release()    
cv2.destroyAllWindows()

In [7]:
#Trainer
if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    print("Training KNN classifier...")
    classifier = train("known_people",
                       model_save_path="{}/models/trained_knn_model.clf".format(curd),
                       n_neighbors=2)
    print("Training complete!")

   

Training KNN classifier...


C:\Users\Choudhary\Anaconda3\envs\mlkit\lib\site-packages\PIL\Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Training complete!


In [7]:
#test Data Creator
import cv2
import numpy as np
import time
curd = os.getcwd()
#name = input()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()
count=0
while count<4:
    curtime = time.strftime("%Y_%m_%d-%H_%M_%S")
    rat, frame = cap.read()
    cv2.imwrite("{}/test/{}.jpg".format(curd,curtime), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


cap.release()    
cv2.destroyAllWindows()

In [15]:
#Still Image Tester
curd = os.getcwd()
for image_file in os.listdir(r"{}\test".format(curd)):
    full_file_path = os.path.join(r"{}\test".format(curd), image_file)

    print("Looking for faces in {}".format(image_file))
    frame = cv2.imread(full_file_path,-1)

    predictions = predict(frame, model_path=r"{}\models\trained_knn_model.clf".format(curd))

    for name, (top, right, bottom, left) in predictions:
        print("- Found {} at ({}, {})".format(name, left, top))
        names.append(name)
    # Display results overlaid on an image
    #show_prediction_labels_on_image(frame, predictions)
    final_img = show_prediction_labels_on_image(frame, predictions)
    cv2.imshow("X",final_img)
    cv2.waitKey(1)
cv2.destroyAllWindows()

Looking for faces in 0.jpg
- Found Shantnu Agarwal at (253, 225)
Looking for faces in 1.jpg
- Found yo at (357, 276)
Looking for faces in 2.jpg
Looking for faces in 2019_03_31-00_37_45.jpg
- Found yash at (259, 235)
Looking for faces in 2019_03_31-00_37_46.jpg
- Found yash at (259, 235)
Looking for faces in 2019_03_31-00_37_47.jpg
- Found yash at (259, 235)
- Found yo at (0, 285)
Looking for faces in 2019_03_31-00_37_48.jpg
- Found yash at (259, 235)
Looking for faces in 3.jpg
Looking for faces in Dx.jpg
- Found DiCap at (167, 39)
Looking for faces in Jx.jpg
- Found Jason at (46, 55)
Looking for faces in momo.jpg
- Found Jason at (8, 32)
- Found unknown at (117, 122)
Looking for faces in XX.jpg
- Found unknown at (33, 62)
- Found Jason at (89, 30)
Looking for faces in Y.jpg
- Found DiCap at (1526, 666)
- Found unknown at (913, 82)
- Found yo at (855, 626)
- Found unknown at (325, 641)
- Found Jason at (268, 96)
- Found unknown at (1491, 133)


In [18]:
#Live Video tester
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()

while(True):
    rat, frame = cap.read()
    predictions = predict(frame, model_path="{}/models/trained_knn_model.clf".format(curd))
    for name, (top, right, bottom, left) in predictions:
        print("- Found {} at ({}, {})".format(name, left, top))
        names.append(name)

        # Display results overlaid on an image
    show_img = show_prediction_labels_on_image(frame, predictions)
    cv2.imshow('img',show_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()    
cv2.destroyAllWindows()

- Found yash at (282, 223)
- Found yash at (270, 199)
- Found Shantnu Agarwal at (275, 216)
- Found Shantnu Agarwal at (270, 211)
- Found yash at (270, 211)
- Found Shantnu Agarwal at (270, 211)
- Found Shantnu Agarwal at (270, 211)
- Found Shantnu Agarwal at (270, 211)
- Found yash at (259, 211)
- Found Shantnu Agarwal at (270, 211)
- Found Shantnu Agarwal at (294, 211)
- Found Shantnu Agarwal at (282, 211)
- Found yash at (282, 211)
- Found Shantnu Agarwal at (259, 211)


In [19]:
#Attendance System csv maker     
namesD=pd.DataFrame(names, columns=["Names"])
namesD= namesD[namesD.Names!="unknown"]

attendance= pd.DataFrame(namesD.iloc[:,0].value_counts())
attendance.rename(index=str,columns={'Names': 'Count'},inplace=True)
attendance["Present"] =0

attendance["Count"][0] > 5
for i in range(attendance.shape[0]):
    if(attendance["Count"][i] > 5):
        attendance["Present"][i] =1


attendance_final=attendance.drop(['Count'],axis=1)
attendance_final
#attendance_final.to_csv('Attendance.csv')

,Present
Shantnu Agarwal,1
yash,1
Jason,0
yo,0
DiCap,0
